In [1]:
# import packages
import numpy as np
import pylab as py
import pdb
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# import nbody6tools reader to read data
from nbody6tools import Reader

# import nbody62spisea to retrieve star ages
from nbody62spisea import converter

/home/wyz5rge/.local/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /scratch/wyz5rge/models/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


## Set up parameters, create isochrone grid

In [2]:
# define isochrone parameters
AKs = 0
dist = 410
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F182M', 'jwst,F200W']
filters = ['m_jwst_F182M', 'm_jwst_F200W']
iso_dir = 'isochrones/'
metallicity = 0

# Select nbody6 simulation and snapshot, then convert to physical units and save as a cluster table which SPISEA can interface with

In [3]:
# read nbody data
path_to_data = '../../../../../../storage/jfarias/Data/Orion/newbinaries/M3000sigma1.0/sfeff001_00/'
snapshot = Reader.read_snapshot(path_to_data, time = 1.5)

# convert snapshot values from nbody units to astrophysical, extract masses
snapshot.to_physical()
cluster_table = converter.to_spicea_table(snapshot)

print(cluster_table)

        mass         isMultiple        compMass             systemMass               e                 log_a                age         
-------------------- ---------- ---------------------- -------------------- -------------------- ------------------ --------------------
 0.19276501331696316        0.0                     []  0.19276501331696316                  nan                nan   1.2059301009070262
 0.20461356920963225        0.0                     []  0.20461356920963225                  nan                nan   0.9443458824697822
  0.6567056965218252        0.0                     []   0.6567056965218252                  nan                nan   1.3725151375241318
0.024930509342311735        0.0                     [] 0.024930509342311735                  nan                nan   1.3725151375241318
                 ...        ...                    ...                  ...                  ...                ...                  ...
   0.073862193837076        1.0  [0.03959

# Read ages of stars, identify bracketing ages for isochrone generation
Also save mass of stars

In [4]:
masses = cluster_table['mass']
level_ages = cluster_table['age']

for i in range(len(level_ages)):
    level_ages[i] = level_ages[i] * 1e6

# convert log ages from log(Myr) to log(yr)
log_ages = np.log10(level_ages)

# print(log_ages)
print("Min log age = " + str(min(log_ages)))
print("Max log age = " + str(max(log_ages)))

# print(level_ages)
print("Min level age = " + str(min(level_ages)))
print("Max level age = " + str(max(level_ages)))

Min log age = 3.5413705432610474
Max log age = 6.177169949959374
Min level age = 3478.3280839685294
Max level age = 1503730.297088623


# Identify binary stars in cluster table, save mass of companion stars

Cluster table always lists binaries last, so finding the index of the first binary suffices

In [5]:
# find index of first binary, create array of companion masses
first_binary = 0
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        first_binary = i
        break
# print(first_binary)

compMasses = []
for i in range(len(cluster_table)):
    if cluster_table['isMultiple'][i] == 1.0:
        compMasses.append(cluster_table['compMass'][i][0])
# print(compMasses)

# Generate isochrones using parameter and age arrays

In [6]:
increment = 0.5e6
start = (min(level_ages) // increment) * increment
end = (max(level_ages) // increment) * increment + 2 * increment

# Create age arrays
level_age_arr = np.arange(start, end, increment)
log_age_arr = np.log10(level_age_arr)

print(log_age_arr)
print(level_age_arr)

[      -inf 5.69897    6.         6.17609126 6.30103   ]
[      0.  500000. 1000000. 1500000. 2000000.]


/tmp/ipykernel_46754/2330829363.py:7: RuntimeWarning: divide by zero encountered in log10
  log_age_arr = np.log10(level_age_arr)


In [7]:
# create isochrone grid - if this is the first time, then this is going to take an hour lmfao
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

ValueError: math domain error

In [ ]:
# identify array indeces, create array for labels and colors
idx_arr = range(0, len(log_age_arr))
if len(idx_arr) > 3:
    idx_arr = idx_arr[::2]

watts_to_lsun = 1.0 / (3.846e26) # conversion factor for watts to Lsun
    
# Plot HR Diagram
fig, axes = py.subplots(figsize=(15, 10))
py.subplot(1, 2, 1)
for i in range(len(log_age_arr)):
    data_iso_Lsun = instances[i].points['L'] * watts_to_lsun
    if i == 0:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'red', label=level_age_arr[i])
    
py.xlabel('Effective Temperature')
py.gca().invert_xaxis()
py.ylabel('Luminosity')
py.yscale('log')
py.legend()

# Use generated SPISEA isochrones to find interpolated magnitudes using the age and mass of each star

Stars outside of the mass range of isochrones are thrown out and not included

In [ ]:
import sys
sys.path.append('/scratch/wyz5rge/synthetic-hr/')
import interpolator as interpolator

level_ages_myr = level_ages / 1e6
# print(level_ages_myr)

# create array of stars
stars = np.empty(len(cluster_table), dtype=object)

# perform interpolation for each star
for i in range(len(stars)):
    # print(str(i) + " " + str(level_ages_myr[i]) + " " + str(masses[i]))
    stars[i] = interpolator.interpolate(level_ages_myr[i], masses[i], instances, log_age_arr, filters)
    
# convert luminosity values to solar luminosities
watts_to_lsun = 1.0 / (3.846e26) # conversion factor for watts to Lsun

for i in range(len(stars)):
    if stars[i] is None:
        continue
    stars[i][0] = stars[i][0] * watts_to_lsun
    
# for i in range(len(stars)):
    # print(stars[i])

# Repeat interpolation on companion stars

Companion stars assumed to be the same age as primary star within system

In [ ]:
# perform interpolation on companion stars
companions = np.empty(len(compMasses), dtype=object)

for i in range(len(companions)):
    if stars[i + first_binary] is None:
        continue
    companions[i] = interpolator.interpolate(level_ages_myr[i + first_binary], compMasses[i], instances, log_age_arr, filters)

for i in range(len(companions)):
    if companions[i] is None:
        continue
    companions[i][0] = companions[i][0] * watts_to_lsun


# Superimpose primary stars and companion stars against isochrones separately

In [ ]:
# plot primaries and companions separately
fig, axes = py.subplots(figsize=(15, 10))

# plot primaries
py.subplot(1, 2, 1)
for i in range(len(log_age_arr)):
    data_iso_Lsun = instances[i].points['L'] * watts_to_lsun
    if i == 0:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'red', label=level_age_arr[i])
    
for i in range(0, first_binary):
    if stars[i] is None:
        continue
    py.plot(stars[i][1], stars[i][0], marker='o', markersize=1, color='k')    
    
py.xlabel('Effective Temperature')
py.gca().invert_xaxis()
py.ylabel('Luminosity')
py.yscale('log')
py.legend()

# plot companions
py.subplot(1, 2, 2)
for i in range(len(log_age_arr)):
    data_iso_Lsun = instances[i].points['L'] * watts_to_lsun
    if i == 0:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'red', label=level_age_arr[i])
        
for i in range(len(companions)):
    if companions[i] is None:
        continue
    py.plot(companions[i][1], companions[i][0], marker='o', markersize=1, color='k')
    
py.xlabel('Effective Temperature')
py.gca().invert_xaxis()
py.ylabel('Luminosity')
py.yscale('log')
py.legend()

# Convert binary stars into unresolved binaries
## Also compute flux ratios of stars and generate histogram
Magnitudes of primary and secondary are converted into flux, which is used to find combined magnitude

In [ ]:
# combine magnitudes on binary stars
unresolved_binaries = np.empty(len(companions), dtype=object)

for i in range(len(companions)):
    if stars[i + first_binary] is None or companions[i] is None:
        continue
    
    lum = stars[i + first_binary][0] + companions[i][0]
    teff = (stars[i + first_binary][1] + companions[i][1]) / 2
    
    unresolved_binaries[i] = [lum, teff]

# Generate CMD using star array, plus unresolved binaries
Compare to CMD generated by simply multiplying each star's flux by 2

In [ ]:
fig, axes = py.subplots(figsize=(15, 10))

py.subplot(1, 2, 1)

for i in range(len(log_age_arr)):
    data_iso_Lsun = instances[i].points['L'] * watts_to_lsun
    if i == 0:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'lightcoral', label=level_age_arr[i])
    elif i == len(log_age_arr) - 1:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'darkred', label=level_age_arr[i])
    else:
        py.plot(instances[i].points['Teff'], data_iso_Lsun, 'red', label=level_age_arr[i])  
    
# plot single stars first
for i in range(0, first_binary):
    if stars[i] is None:
        continue
    py.plot(stars[i][1], stars[i][0], marker='o', markersize=1, color='k')

# plot unresolve binaries
for i in range(len(unresolved_binaries)):
    if unresolved_binaries[i] is None:
        continue
    py.plot(stars[first_binary + i][1], unresolved_binaries[i][0], marker='o', markersize=1, color='green')
    
py.xlabel('Effective Temperature')
py.gca().invert_xaxis()
py.ylabel('Luminosity')
py.yscale('log')
py.legend()

In [ ]:
lums = []

count = 0
for i in range(0, first_binary):
    if stars[i] is None:
        continue
    lums.append(stars[i][0])
    count = count + 1

for i in range(len(unresolved_binaries)):
    if unresolved_binaries[i] is None:
        continue
    lums.append(unresolved_binaries[i][0])
    count = count + 1

print(count)

std = np.std(lums)
print("Standard deviation: " + str(std))

In [ ]:
temperature_bins = np.arange(3000, 3700, 100)
standard_deviations = {temp: [] for temp in temperature_bins}

# Populate temperature bins and calculate standard deviations
for temp in temperature_bins:
    luminosities = []
    for i in range(0, first_binary):
        if stars[i] is None:
            continue
        if temp <= stars[i][1] < temp + 100:
            luminosities.append(stars[i][0])
    for i in range(len(unresolved_binaries)):
        if unresolved_binaries[i] is None or stars[i + first_binary] is None:
            continue
        if temp <= stars[i + first_binary][1] < temp + 100:
            luminosities.append(unresolved_binaries[i][0])
    
    standard_deviation = np.std(luminosities)
    standard_deviations[temp] = standard_deviation

# Print standard deviations for each temperature bin
for temp, std_dev in standard_deviations.items():
    print(f"Temperature: {temp} K, Standard Deviation: {std_dev}")
            